In [1]:
# Decoding Attributes
# setting up dictionaries for each attribute
hairs = {
    0: 'wavy',
    1: 'curly',
    2: 'beanie',
    3: 'cap'
}
colors = {
    0: 'red',
    1: 'blue',
    2: 'yellow',
    3: 'green'
}
accessories = {
    0: 'sneakers',
    1: 'bowtie',
    2: 'sunglasses',
    3: 'scarf'
}
socials = {
    0: 'tiktok',
    1: 'instagram',
    2: 'discord',
    3: 'snapchat'
}


# function to decode the attributes
def zbini_attrs(type_id):
    # checking if the type_id is valid
    if type_id < 0 or type_id > 255:
        return None
    # decoding the attributes
    # hair_id is the result of type_id // 64
    hair = hairs[type_id // 64]
    # color_id is the result of (type_id % 64) // 16
    color = colors[(type_id % 64) // 16]
    # accessory_id is the result of (type_id % 16) // 4
    accessory = accessories[(type_id % 16) // 4]
    # social_id is the result of type_id % 4
    social = socials[type_id % 4]
    return hair, color, accessory, social

print(zbini_attrs(0))
print(zbini_attrs(108))
print(zbini_attrs(148))
print(zbini_attrs(248))
# print(zbini_attrs(96))
# print(zbini_attrs(351))

('wavy', 'red', 'sneakers', 'tiktok')
('curly', 'yellow', 'scarf', 'tiktok')
('beanie', 'blue', 'bowtie', 'tiktok')
('cap', 'green', 'sunglasses', 'tiktok')


In [2]:
# Checking Group Validity
def valid_study_group(zbinis, group):
    # checking if the length of group is valid
    if len(group) < 3 or len(group) > 4:
        return False, None
    # get all zbini form the group and zbinis
    zbini = [zbinis[i] for i in group]
    # get the index from zbini
    index_members = [(i[0]) for i in zbini]
    # get the attributes from the index members
    attrs = [zbini_attrs(i) for i in index_members]
    # checking if the attributes are unique
    if len(attrs) != len(set(attrs)):
        return False, None
    # put the first set of attributes in the intersection
    intersection = set(zbini[0][1])
    # do for loop in the rest of the zbini
    for i in zbini[1:]:
        # obtain the intersection of the attributes
        intersection = intersection.intersection(set(i[1]))
    # checking if the intersection is empty
    if len(intersection) == 0:
        return False, None
    return True, len(intersection)
    
example_zbinis = [(0, ['FoC', 'Calc 1', 'Logic']), (108, ['FoA', 'Calc 2', 'Logic']), (148, ['FoC', 'Calc 1', 'Logic']), (248, ['FoC', 'Calc 1', 'Logic']), (0, ['Calc 2', 'History', 'Politics']), (108, ['FoC', 'Calc 1', 'Logic'])]
print(valid_study_group(example_zbinis, (0, 1, 3)))
# (True, 1)
print(valid_study_group(example_zbinis, (0, 1, 5)))
# (False, None)
print(valid_study_group(example_zbinis, (0, 1, 2, 3)))
# (True, 1)
print(valid_study_group(example_zbinis, (1, 2, 3, 4)))
# (False, None)

(True, 1)
(False, None)
(True, 1)
(False, None)


In [3]:
# Possible Study Groups
# import the combinations function from itertools
from itertools import combinations


def possible_study_groups(zbinis):
    # create a list for storing valid groups
    groups = []
    # generate indexes from the length of zbinis
    possible_index = [i for i in range(len(zbinis))]
    # generate all possible groups of three members
    possible_groups_three = list(combinations(possible_index, 3))
    # generate all possible groups of four members
    possible_groups_four = list(combinations(possible_index, 4))
    # combine the possible groups
    possible_groups = possible_groups_four + possible_groups_three
    for i in possible_groups:
        # check if the group is valid using the valid_study_group function
        tf, num = valid_study_group(zbinis, i)
        if tf:
            # calculate the score
            score = num * 3 + 1 if len(i) == 3 else num * 3 + 2
            # append the group and score to the groups list
            groups.append((i, score))
    return sorted(groups, key=lambda x: x[1], reverse=True)

print(possible_study_groups([(198, ['FoC']), (138, ['FoC', 'Calc 1']), (14, ['FoC', 'Calc 1']), (66, ['Calc 1'])]))
# [((0, 1, 2), 4), ((1, 2, 3), 4)]
print(possible_study_groups([(198, ['FoC']), (138, ['FoC', 'Calc 1']), (14, ['FoC']), (66, ['FoC'])]))
# [((0, 1, 2, 3), 5), ((0, 1, 2), 4), ((0, 1, 3), 4), ((0, 2, 3), 4), ((1, 2, 3), 4)]
print(possible_study_groups([(198, ['FoC']), (138, ['Calc 1']), (14, ['Calc 1']), (66, ['FoC'])]))
# []

[((0, 1, 2), 4), ((1, 2, 3), 4)]
[((0, 1, 2, 3), 5), ((0, 1, 2), 4), ((0, 1, 3), 4), ((0, 2, 3), 4), ((1, 2, 3), 4)]
[]


In [5]:
# Allocating Study Groups
# define a function to generate all possible combinations
# and return the maximum length combinations
def generate_combinations(possible_groups):
    """
    :param possible_groups: a list from the possible_study_groups function 
    :return: the maximum length combinations from valid groups
    """
    all_combinations = []
    # iterate through the possible groups
    for i in range(len(possible_groups)):
        all_combination = [possible_groups[i]]
        for j in range(i + 1, len(possible_groups)):
            temp = []
            for k in all_combination:
                temp += k[0]
            # check if the intersection of the group and the union is empty
            inter = set(temp).intersection(set(possible_groups[j][0]))
            if len(inter) == 0:
                all_combination.append(possible_groups[j])
        # store the combinations in all_combinations
        all_combinations.append(all_combination)
    # calculate the maximum length of all_combinations
    max_length = 0
    for i in all_combinations:
        max_length = max(sum([(len(j[0])) for j in i]), max_length)
    max_length_combinations = []
    # get the combinations with the maximum length
    for i in all_combinations:
        if sum([len(j[0]) for j in i]) == max_length:
            max_length_combinations.append(i)
    return max_length_combinations

def alloc_study_groups(zbinis):
    groups = []
    # get the possible groups using the possible_study_groups function
    possible_groups = possible_study_groups(zbinis)
    # get the maximum length combinations
    max_length_combinations = generate_combinations(possible_groups)
    # check if the max_length_combinations is empty
    if not max_length_combinations:
        return groups
    # calculate the maximum score
    max_score = 0
    for i in max_length_combinations:
        max_score = max(sum([(j[1]) for j in i]), max_score)
    max_score_combinations = []
    # get the combinations with the maximum score
    for i in max_length_combinations:
        if sum([j[1] for j in i]) == max_score:
            max_score_combinations.append(i)
    # sorted by minimum index etc.
    for i in range(len(max_score_combinations)):
        t = []
        for j in range(len(max_score_combinations[i])):
            t += max_score_combinations[i][j][0]
        max_score_combinations[i].append(sorted(set(t)))
    max_score_combinations.sort(key=lambda x: x[-1])
    # get the first group of max_score_combinations
    # sorted by score
    final = sorted(max_score_combinations[0][:-1], key=lambda x: -x[-1])
    for i in final:
        groups.append(i[0])
    return groups
    
print(alloc_study_groups([(198, ['FoC']), (198, ['FoC']), (18, ['FoC']), (19, ['FoC'])]))
# # [(0, 2, 3)]
print(alloc_study_groups([(198, ['FoC']), (138, ['FoC', 'Calc 1']), (14, ['FoC', 'Calc 1']), (66, ['FoC', 'Calc 1'])]))
# [(0, 1, 2, 3)]
print(alloc_study_groups([(198, ['FoC', 'Logic']), (138, ['Calc 1']), (14, ['Calc 1']), (66, ['FoC', 'Logic']), (10, ['FoC', 'Logic']), (142, ['FoC', 'Logic']), (66, ['Calc 1']), (18, ['FoC']), (19, ['FoC'])]))
# [(0, 3, 4, 5), (1, 2, 6)]

[(0, 2, 3)]
[(0, 1, 2, 3)]
[(0, 3, 4), (1, 2, 6), (5, 7, 8)]


In [ ]:
# Allocating Study Groups
# define a function to generate all possible combinations
# and return the maximum length combinations
def generate_combinations(tuples):
    """
    :param possible_groups: a list from the possible_study_groups function 
    :return: the maximum length combinations from valid groups
    """
    all_combinations = []
    # iterate through the possible groups
    for i in range(len(tuples)):
        all_combination = [tuples[i]]
        for j in range(i + 1, len(tuples)):
            temp = []
            for k in all_combination:
                temp += k[0]
            # check if the intersection of the group and the union is empty
            inter = set(temp).intersection(set(tuples[j][0]))
            if len(inter) == 0:
                all_combination.append(tuples[j])
        # store the combinations in all_combinations
        all_combinations.append(all_combination)
    # calculate the maximum length of all_combinations
    max_length = 0
    # get the combinations with the maximum length
    for i in all_combinations:
        max_length = max(sum([(len(j[0])) for j in i]), max_length)
    # get the combinations with the maximum length
    all_combinations = [i for i in all_combinations 
                        if sum([len(j[0]) for j in i]) == max_length]
    return all_combinations
def alloc_study_groups(zbinis):
    """
    
    :param zbinis: a list from system
    :return: the one of most perfect study groups
    """
    groups = []
    # get the possible groups using the possible_study_groups function
    possible_groups = possible_study_groups(zbinis)
    # get the maximum length combinations
    all_combinations = generate_combinations(possible_groups)
    # check if the max_length_combinations is empty
    if not all_combinations:
        return groups
    max_score = 0
    # calculate the maximum score
    for i in all_combinations:
        max_score = max(sum([(j[1]) for j in i]), max_score)
    all_combinations = [i for i in all_combinations 
                        if sum([j[1] for j in i]) == max_score]
    # sorted by minimum index etc.
    for i in range(len(all_combinations)):
        t = []
        for j in range(len(all_combinations[i])):
            t += all_combinations[i][j][0]
        all_combinations[i].append(sorted(set(t)))
    all_combinations.sort(key=lambda x: x[-1])
    # get the first group of max_score_combinations
    # sorted by score
    for i in sorted(all_combinations[0][:-1], 
                    key=lambda x: x[-1], reverse=True):
        groups.append(i[0])
    return groups

print(alloc_study_groups([(198, ['FoC']), (198, ['FoC']), (138, ['FoC']), (14, ['FoC'])]))
# [(0, 2, 3)]
print(alloc_study_groups([(198, ['FoC']), (138, ['FoC', 'Calc 1']), (14, ['FoC', 'Calc 1']), (66, ['FoC', 'Calc 1'])]))
# [(0, 1, 2, 3)]
print(alloc_study_groups([(198, ['FoC', 'Logic']), (138, ['Calc 1']), (14, ['Calc 1']), (66, ['FoC', 'Logic']), (10, ['FoC', 'Logic']), (142, ['FoC', 'Logic']), (66, ['Calc 1'])]))
# [(0, 3, 4, 5), (1, 2, 6)]